In [ ]:
# pip install torch torchvision
# pip install -U openmim
# pip install mmengine
# pip install mmcv
# mim install mmdet
# git clone https://github.com/open-mmlab/mmdetection.git
# cd mmdetection
# pip install -v -e .

In [4]:
! pip install mrcfile
! pip install -U cryoet-data-portal
! pip install matplotlib
! pip install groundingdino-py
! pip install scikit-learn
! pip install awscli
! pip install --upgrade urllib3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for groundingdino-py: filename=groundingdino_py-0.4.0-py2.py3-none-any.whl size=88749 sha256=2888057a591257d479be8eab32c8f7463ddf4969c210ff6f7a9a367179e93170
  Stored in directory: /root/.cache/pip/wheels/72/25/30/97b491abad279d329c62bef1e91bc56bf2fd40b22281068e1d
Successfully built groundingdino-py
   ━━━━━━━

In [5]:
# Standard library imports
import io
import json
import os
import subprocess
from pathlib import Path

# Third party imports
import cryoet_data_portal as portal
import matplotlib.pyplot as plt
import mrcfile
import numpy as np
import requests
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split


# Instantiate a client, using the data portal GraphQL API by default
client = portal.Client()

# runs also have s3_prefix where the run data is stored, might be helpful later

def find_dataset_by_id(dataset_id):
    datasets = portal.Dataset.find(client, [portal.Dataset.id == dataset_id])
    return datasets[0] # seems like this is always 1 for the most part


def get_dataset_to_runs_for_dataset_id(dataset_id):
    dataset = find_dataset_by_id(dataset_id)
    return {dataset.id :[run.name for run in dataset.runs]}


def get_run_to_tomograms_for_dataset_id(dataset_id):
    dataset = find_dataset_by_id(dataset_id)
    runs = dataset.runs
    run_to_tomograms = {}
    for run in runs:
        run_to_tomograms[run.name] = run.tomograms
    return run_to_tomograms


def get_annotations_for_tomogram(tomogram):
    annotations = portal.Annotation.find(client, [portal.Tomogram.id == tomogram.id])
    return annotations


def download_mrc_for_tomogram(dataset_id, tomogram):
    """Download the MRC file for a given tomogram"""
    # TODO: download to the run name folder generated from the aws sync commands
    # new filename should be {tomogram.id}.mrc
    url = tomogram.https_mrc_file
    dir_name = f"{dataset_id}_{tomogram.run.name}_{tomogram.id}"
    os.makedirs(dir_name, exist_ok=True)
    local_file = f"{dir_name}/{tomogram.voxel_spacing}_downloaded.mrc"
    response = requests.get(url)
    with open(local_file, 'wb') as f:
       f.write(response.content)


def get_tomogram_to_annotation_for_run_id(tomograms: list[portal.Tomogram]):
    return {
        tomogram.id: get_annotations_for_tomogram(tomogram)
        for tomogram in tomograms
    }


def visualize_slice_and_save(mrc_path, z_slice: int, tomogram_id):
    with mrcfile.open(mrc_path) as mrc:
       slice = mrc.data[z_slice, :, :]
       plt.figure(figsize=(10,10))
       plt.imshow(slice, cmap='gray')
       plt.colorbar()
       plt.title(f'Tomogram Slice')
       plt.show()
       slice_norm = (slice - np.min(slice)) / (np.max(slice) - np.min(slice))
       plt.imsave(f'{tomogram_id}_{z_slice}_slice.png', slice_norm, cmap='gray', vmin=0, vmax=1)


def save_mrc_slice(mrc_path: str, z_slice: int, tomogram_id: str,
                  voxel_spacing: float = None) -> None:
    """Save a normalized slice from an MRC file"""
    with mrcfile.open(mrc_path) as mrc:
        slice = mrc.data[z_slice, :, :]
        slice_norm = (slice - np.min(slice)) / (np.max(slice) - np.min(slice))
        if voxel_spacing:
            plt.figure(figsize=(10, 10), dpi=300)
            # By multiplying the pixel dimensions (slice.shape) by voxel_spacing,
            # we convert from pixel coordinates to physical units (e.g., nanometers),
            # making the scale bars and measurements scientifically meaningful.
            plt.imshow(slice_norm, cmap='gray', extent=[0, slice.shape[1]*voxel_spacing,
                      slice_norm.shape[0]*voxel_spacing, 0])
        else:
            plt.figure(figsize=(10, 10), dpi=300)
            plt.imshow(slice_norm, cmap='gray')

        plt.savefig(f'{tomogram_id}_{z_slice}_slice.png', bbox_inches='tight', dpi=300)
        plt.close()

def process_and_save_all_mrc_layers(mrc_path: str):
    # Get directory containing the MRC file
    mrc_dir = os.path.dirname(mrc_path)
    if not mrc_dir:  # If empty string (current directory)
        mrc_dir = '.'
    voxel_spacing = os.path.basename(mrc_path).replace('_downloaded.mrc', '')
    with mrcfile.open(mrc_path) as mrc:
        num_layers = mrc.data.shape[0]
        for z in range(num_layers):
            slice = mrc.data[z, :, :]
            slice_norm = (slice - np.min(slice)) / (np.max(slice) - np.min(slice))
            # Convert to 8-bit (0-255) for PNG
            slice_norm = (slice_norm * 255).astype(np.uint8)
            img = Image.fromarray(slice_norm)
            img.save(f'{mrc_dir}/{voxel_spacing}_{z}_slice.png')
            print(f"Processed layer {z}/{num_layers-1}")

def sync_annotations_cmds_for_datasets(dataset_to_runs: dict[str, list[str]], run_to_tomograms: dict[str, list[portal.Tomogram]]):
    commands = set()
    for dataset_id, run_names in dataset_to_runs.items():
        for run_name in run_names:
            # The * wildcard will work with aws s3 sync command
            # It will match any VoxelSpacing directory like VoxelSpacing10.012, VoxelSpacing14.848 etc.
            tomograms = run_to_tomograms[run_name]
            for tomogram in tomograms:
                cmd = f"aws s3 --no-sign-request sync s3://cryoet-data-portal-public/{dataset_id}/{run_name}/Reconstructions/VoxelSpacing{tomogram.voxel_spacing}/Annotations {run_name}/Annotations"
                commands.add(cmd)
    return commands

In [ ]:
project_id = 10440
tomograms_for_10440 = get_run_to_tomograms_for_dataset_id(project_id)
tomograms_for_10440

{'TS_5_4': [<cryoet_data_portal._models.Tomogram at 0x7efd7fd115a0>,
 'TS_69_2': [<cryoet_data_portal._models.Tomogram at 0x7efd7fd36a40>,
 'TS_6_4': [<cryoet_data_portal._models.Tomogram at 0x7efd4e05e590>,
 'TS_6_6': [<cryoet_data_portal._models.Tomogram at 0x7efd4e100910>,
 'TS_73_6': [<cryoet_data_portal._models.Tomogram at 0x7efd4e102bf0>,
 'TS_86_3': [<cryoet_data_portal._models.Tomogram at 0x7efd4df38e20>,
 'TS_99_9': [<cryoet_data_portal._models.Tomogram at 0x7efd4df3b2e0>,
  <cryoet_data_portal._models.Tomogram at 0x7efd4df3b100>]}

In [ ]:
dataset_to_runs_10440 = get_dataset_to_runs_for_dataset_id(project_id)
sync_cmds = sync_annotations_cmds_for_datasets(dataset_to_runs_10440, tomograms_for_10440)

for cmd in sync_cmds:
    subprocess.run(cmd.split())

In [ ]:
sync_cmds

{'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_5_4/Reconstructions/VoxelSpacing10.012/Annotations TS_5_4/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_69_2/Reconstructions/VoxelSpacing10.012/Annotations TS_69_2/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_6_4/Reconstructions/VoxelSpacing10.012/Annotations TS_6_4/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_6_6/Reconstructions/VoxelSpacing10.012/Annotations TS_6_6/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_73_6/Reconstructions/VoxelSpacing10.012/Annotations TS_73_6/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_86_3/Reconstructions/VoxelSpacing10.012/Annotations TS_86_3/Annotations',
 'aws s3 --no-sign-request sync s3://cryoet-data-portal-public/10440/TS_99_9/Reconstructions/VoxelSpacing10.012/Annotations TS_99_

In [ ]:
for run_name, tomograms in tomograms_for_10440.items():
    for tomogram in tomograms:
        download_mrc_for_tomogram(project_id, tomogram)

In [ ]:
process_and_save_all_mrc_layers("10440_TS_99_9_17042/10.012_downloaded.mrc")

Processed layer 0/183
Processed layer 1/183
Processed layer 2/183
Processed layer 3/183
Processed layer 4/183
Processed layer 5/183
Processed layer 6/183
Processed layer 7/183
Processed layer 8/183
Processed layer 9/183
Processed layer 10/183
Processed layer 11/183
Processed layer 12/183
Processed layer 13/183
Processed layer 14/183
Processed layer 15/183
Processed layer 16/183
Processed layer 17/183
Processed layer 18/183
Processed layer 19/183
Processed layer 20/183
Processed layer 21/183
Processed layer 22/183
Processed layer 23/183
Processed layer 24/183
Processed layer 25/183
Processed layer 26/183
Processed layer 27/183
Processed layer 28/183
Processed layer 29/183
Processed layer 30/183
Processed layer 31/183
Processed layer 32/183
Processed layer 33/183
Processed layer 34/183
Processed layer 35/183
Processed layer 36/183
Processed layer 37/183
Processed layer 38/183
Processed layer 39/183
Processed layer 40/183
Processed layer 41/183
Processed layer 42/183
Processed layer 43/18

In [27]:
import requests
import numpy as np
import subprocess
import io
import matplotlib.pyplot as plt
import glob
from PIL import ImageDraw
import json
from PIL import Image
import os
from pathlib import Path


def read_points_file(file_path: str) -> list:
    with open(file_path, 'r') as f:
        return [json.loads(line) for line in f]

def annotate_slice_with_points(slice_path: str, points: list, z_index: int, color: str, box_size: int = 30):
    if os.path.exists(slice_path.replace('_slice.png', '_slice_annotated.png')):
        img = Image.open(slice_path.replace('_slice.png', '_slice_annotated.png')).convert('RGB')
    else:
        img = Image.open(slice_path).convert('RGB')

    draw = ImageDraw.Draw(img)
    for point in points:
        loc = point['location']
        if abs(loc['z'] - z_index) <= 0.5:
            x, y = int(loc['x']), int(loc['y'])
            half_size = box_size // 2
            draw.rectangle([x-half_size, y-half_size, x+half_size, y+half_size],
                         outline=color, width=2)
    return img


# def annotate_slice_with_points(slice_path: str, points: list, z_index: int, radius: int = 5):
#     """Draw points on a slice image if they're within ±0.5 of the z_index"""
#     img = Image.open(slice_path)
#     draw = ImageDraw.Draw(img)

#     for point in points:
#         loc = point['location']
#         if abs(loc['z'] - z_index) <= 0.5:  # Point is on this slice
#             x, y = int(loc['x']), int(loc['y'])
#             draw.ellipse([x-radius, y-radius, x+radius, y+radius],
#                         fill='red', outline='white')

#     return img


def process_all_slices(points_file: str, base_dir: str, color: str):
    points = read_points_file(points_file)
    pattern = os.path.join(base_dir, '*_slice.png')
    slice_files = glob.glob(pattern)

    for slice_file in slice_files:
        if any(x in slice_file for x in ["39", "40", "35", "31", "32"]):
            z = int(slice_file.split('_')[-2])
            annotated = annotate_slice_with_points(slice_file, points, z, color)
            annotated.save(slice_file.replace('_slice.png', '_slice_annotated.png'))

# Usage
process_all_slices("TS_99_9/Annotations/100/ferritin_complex-1.0_point.ndjson", "10440_TS_99_9_17042", "red")
process_all_slices("TS_99_9/Annotations/101/beta_amylase-1.0_point.ndjson", "10440_TS_99_9_17042", "blue")
process_all_slices("TS_99_9/Annotations/102/beta_galactosidase-1.0_point.ndjson", "10440_TS_99_9_17042", "green")
process_all_slices("TS_99_9/Annotations/103/cytosolic_ribosome-1.0_point.ndjson", "10440_TS_99_9_17042", "yellow")
process_all_slices("TS_99_9/Annotations/104/thyroglobulin-1.0_point.ndjson", "10440_TS_99_9_17042", "orange")
process_all_slices("TS_99_9/Annotations/105/pp7_vlp-1.0_point.ndjson", "10440_TS_99_9_17042", "purple")


In [ ]:
# ! rm 10440_TS_99_9_17042/*_annotated.png

In [11]:
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
import json

def create_coco_dataset(image_dir: str, annotation_files: dict):
    """
    Convert tomogram annotations to COCO format

    Args:
        image_dir: Directory containing annotated slice images
        annotation_files: Dict mapping category names to annotation file paths
    """
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Create categories
    for cat_id, category in enumerate(annotation_files.keys(), 1):
        coco_format["categories"].append({
            "id": cat_id,
            "name": category
        })

    # Map category names to IDs
    category_map = {cat["name"]: cat["id"] for cat in coco_format["categories"]}

    # Process images
    image_id = 0
    annotation_id = 0

    for img_path in Path(image_dir).glob("*_slice.png"):
        # Get image dimensions
        img = Image.open(img_path)
        width, height = img.size

        # Add image info
        coco_format["images"].append({
            "id": image_id,
            "file_name": img_path.name,
            "width": width,
            "height": height
        })

        # Get slice number
        z_index = int(img_path.stem.split('_')[-2])

        # Process each category's annotations
        for category, anno_file in annotation_files.items():
            with open(anno_file) as f:
                points = [json.loads(line) for line in f]

            # Add annotations for points on this slice
            for point in points:
                loc = point['location']
                if abs(loc['z'] - z_index) <= 0.5:
                    # Convert center point to bbox [x,y,width,height]
                    box_size = 30  # Same as used in annotation
                    half_size = box_size // 2
                    bbox = [
                        int(loc['x']) - half_size,
                        int(loc['y']) - half_size,
                        box_size,
                        box_size
                    ]

                    coco_format["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": category_map[category],
                        "bbox": bbox,
                        "area": box_size * box_size,
                        "iscrowd": 0
                    })
                    annotation_id += 1

        image_id += 1

    return coco_format

# Usage
annotation_files = {
    "ferritin_complex": "/content/drive/MyDrive/TS_99_9/Annotations/100/ferritin_complex-1.0_point.ndjson",
    "beta_amylase": "/content/drive/MyDrive/TS_99_9/Annotations/101/beta_amylase-1.0_point.ndjson",
    "beta_galactosidase": "/content/drive/MyDrive/TS_99_9/Annotations/102/beta_galactosidase-1.0_point.ndjson",
    "cytosolic_ribosome": "/content/drive/MyDrive/TS_99_9/Annotations/103/cytosolic_ribosome-1.0_point.ndjson",
    "thyroglobulin": "/content/drive/MyDrive/TS_99_9/Annotations/104/thyroglobulin-1.0_point.ndjson",
    "virus": "/content/drive/MyDrive/TS_99_9/Annotations/105/pp7_vlp-1.0_point.ndjson"
}

coco_annotations = create_coco_dataset("/content/drive/MyDrive/10440_TS_99_9_17042", annotation_files)

# Save COCO format annotations
with open('annotations_coco.json', 'w') as f:
    json.dump(coco_annotations, f, indent=2)

In [6]:
from sklearn.model_selection import train_test_split


with open('annotations_coco.json') as f:
    data = json.load(f)

train_imgs, val_imgs = train_test_split(data['images'], test_size=0.2)

# Create train and val annotation files
train_data = data.copy()
train_data['images'] = train_imgs
with open('train_coco.json', 'w') as f:
    json.dump(train_data, f)

val_data = data.copy()
val_data['images'] = val_imgs
with open('val_coco.json', 'w') as f:
    json.dump(val_data, f)



# start finetuning groundingdyno with mmdetection using annotation point coco datasets


---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())
!pip install -U openmim
!mim install "mmengine>=0.7.0"
!mim install "mmcv>=2.0.0,<2.1.0"
#mmcv>=2.0.0rc4,<2.2.0
# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)
#This method has worked for me on google colab.

# Run the training script
# ! python tools/train.py path/to/finetune_config.py


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 520.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=00ecdc3b8271bab9dac1734ab65ea4071cb8c697583372d9eef4e03a5f722066
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing in

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 21.8 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 8.8 MB/s eta 0:00:00
mmcv: 2.0.1


In [8]:
# ! git clone https://github.com/open-mmlab/mmdetection.git
! pip install -v -e /content/drive/MyDrive/mmdetection
# also add CustomCocoDataset to mmdet/datasets and add to __init__

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/drive/MyDrive/mmdetection
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-ohhmtp4_/mmdet.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-ohhmtp4_/mmdet.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-ohhmtp4_/mmdet.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for terminaltables from https://files.pythonhosted.org/packages/c4/fb/ea621e0a19733e01fe4005d46087d383693c0f4a8f824b47d8d4122c87e0/terminaltables-3.1.10-py2.py3-none-any.whl.metadata
  Running setup.py develop for mmdet
    Running command python setup.py develop
    running develop
    /usr/local/lib/python3.10/dist-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
      warnings.warn

In [ ]:
! mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest .

processing rtmdet_tiny_8xb32-300e_coco...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 MiB 26.6 MB/s eta 0:00:00
Successfully downloaded rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth to /content
Traceback (most recent call last):
  File "/usr/local/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/

In [ ]:
! pip install numpy==1.23.5

! cd mmdetection && python demo/image_demo.py demo/demo.jpg ../rtmdet_tiny_8xb32-300e_coco.py --weights ../rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth --device cuda

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 29.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.27.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
pymc 5.18.2 requires rich>=13.7.1, but you have rich 13.4.2 which is 

2024-12-10 20:52:46.257823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 20:52:46.381786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 20:52:46.398457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 20:52:46.472363: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-10 20:52:50.369309: W tensorflow/comp

In [9]:
from transformers import BertConfig, BertModel
from transformers import AutoTokenizer

config = BertConfig.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", add_pooling_layer=False, config=config)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

config.save_pretrained("bert-base-uncased")
model.save_pretrained("bert-base-uncased")
tokenizer.save_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('bert-base-uncased/tokenizer_config.json',
 'bert-base-uncased/special_tokens_map.json',
 'bert-base-uncased/vocab.txt',
 'bert-base-uncased/added_tokens.json',
 'bert-base-uncased/tokenizer.json')

In [10]:
import json

with open('train_coco.json') as f:
    data = json.load(f)
print(f"Images: {len(data['images'])}")
print(f"Annotations: {len(data['annotations'])}")
print(f"Categories: {len(data['categories'])}")

Images: 147
Annotations: 202
Categories: 6


In [13]:
# Check first few image paths exist
import os
print(f"First image path: {data['images'][0]['file_name']}")
print(f"Exists?: {os.path.exists(os.path.join('10440_TS_99_9_17042', data['images'][0]['file_name']))}")

First image path: 10.012_92_slice.png
Exists?: False


In [14]:
print("Sample annotation:")
print(json.dumps(data['annotations'][0], indent=2))

print("\nCategories:")
for cat in data['categories']:
    print(f"{cat['id']}: {cat['name']}")

Sample annotation:
{
  "id": 0,
  "image_id": 2,
  "category_id": 5,
  "bbox": [
    52,
    68,
    30,
    30
  ],
  "area": 900,
  "iscrowd": 0
}

Categories:
1: ferritin_complex
2: beta_amylase
3: beta_galactosidase
4: cytosolic_ribosome
5: thyroglobulin
6: virus


In [15]:
image_ids = set(img['id'] for img in data['images'])
anno_image_ids = set(anno['image_id'] for anno in data['annotations'])
print(f"Missing image IDs: {anno_image_ids - image_ids}")

Missing image IDs: {6, 8, 144, 25, 27, 46, 48, 49, 53, 58, 64, 70, 78, 80, 87, 90, 97, 110, 111, 115, 125, 126}


In [16]:
# Remove annotations without matching images
valid_annotations = [anno for anno in data['annotations'] if anno['image_id'] in image_ids]
data['annotations'] = valid_annotations

# Save fixed data
with open('train_coco.json', 'w') as f:
    json.dump(data, f)

print(f"Annotations after cleanup: {len(valid_annotations)}")

Annotations after cleanup: 159


In [16]:
# Load and fix val data
with open('val_coco.json') as f:
    val_data = json.load(f)

val_image_ids = set(img['id'] for img in val_data['images'])
val_valid_annotations = [anno for anno in val_data['annotations'] if anno['image_id'] in val_image_ids]
val_data['annotations'] = val_valid_annotations

with open('val_coco.json', 'w') as f:
    json.dump(val_data, f)

print(f"Val annotations after cleanup: {len(val_valid_annotations)}")

Val annotations after cleanup: 34


In [12]:
# Add text fields to each annotation
for anno in data['annotations']:
    category_name = next(cat['name'] for cat in data['categories'] if cat['id'] == anno['category_id'])
    anno['text'] = category_name

# Save updated data
with open('train_coco.json', 'w') as f:
    json.dump(data, f)

# Do same for validation
with open('val_coco.json') as f:
    val_data = json.load(f)

for anno in val_data['annotations']:
    category_name = next(cat['name'] for cat in val_data['categories'] if cat['id'] == anno['category_id'])
    anno['text'] = category_name

with open('val_coco.json', 'w') as f:
    json.dump(val_data, f)

In [18]:
with open('train_coco.json') as f:
    data = json.load(f)

# Check first annotation structure
print(json.dumps(data['annotations'][0], indent=2))

# Verify each annotation has required fields
required_fields = ['bbox', 'image_id', 'category_id']
missing = [i for i, anno in enumerate(data['annotations'])
          if not all(field in anno for field in required_fields)]
print(f"Annotations missing required fields: {missing}")

{
  "id": 0,
  "image_id": 2,
  "category_id": 5,
  "bbox": [
    52,
    68,
    30,
    30
  ],
  "area": 900,
  "iscrowd": 0,
  "text": "thyroglobulin"
}
Annotations missing required fields: []


In [13]:
# Update train data
with open('train_coco.json') as f:
    train_data = json.load(f)

# Remove text from images if it exists
for img in train_data['images']:
    if 'text' in img:
        del img['text']

# Add text to each annotation
for ann in train_data['annotations']:
    ann['text'] = "Find ferritin complex, beta amylase, beta galactosidase, cytosolic ribosome, thyroglobulin, and virus"

with open('train_coco.json', 'w') as f:
    json.dump(train_data, f)

# Update validation data
with open('val_coco.json') as f:
    val_data = json.load(f)

# Remove text from images if it exists
for img in val_data['images']:
    if 'text' in img:
        del img['text']

# Add text to each annotation
for ann in val_data['annotations']:
    ann['text'] = "Find ferritin complex, beta amylase, beta galactosidase, cytosolic ribosome, thyroglobulin, and virus"

with open('val_coco.json', 'w') as f:
    json.dump(val_data, f)

In [ ]:
# Check first image path and existence
print("Sample image path:", os.path.join('10440_TS_99_9_17042', data['images'][0]['file_name']))
print("Images exist:", all(os.path.exists(os.path.join('10440_TS_99_9_17042', img['file_name']))
                          for img in data['images']))

Sample image path: 10440_TS_99_9_17042/10.012_5_slice.png
Images exist: True


In [ ]:
# Print first few images and annotations that would be loaded
for i in range(50,55):
    print(f"\nImage {i}:")
    print(f"Path: {data['images'][i]['file_name']}")
    annos = [a for a in data['annotations'] if a['image_id'] == data['images'][i]['id']]
    print(f"Number of annotations: {len(annos)}")
    for a in annos:
        print(f"Category: {a['category_id']}, BBox: {a['bbox']}")


Image 50:
Path: 10.012_101_slice.png
Number of annotations: 1
Category: 4, BBox: [316, 531, 30, 30]

Image 51:
Path: 10.012_76_slice.png
Number of annotations: 4
Category: 1, BBox: [30, 181, 30, 30]
Category: 5, BBox: [535, 447, 30, 30]
Category: 6, BBox: [293, 33, 30, 30]
Category: 6, BBox: [13, 395, 30, 30]

Image 52:
Path: 10.012_16_slice.png
Number of annotations: 0

Image 53:
Path: 10.012_169_slice.png
Number of annotations: 0

Image 54:
Path: 10.012_114_slice.png
Number of annotations: 5
Category: 2, BBox: [265, 561, 30, 30]
Category: 4, BBox: [477, 543, 30, 30]
Category: 4, BBox: [273, 276, 30, 30]
Category: 5, BBox: [107, 186, 30, 30]
Category: 5, BBox: [201, 17, 30, 30]


In [ ]:
print("Required GroundingDINO fields:")
print("\nImage format:")
print(json.dumps(data['images'][0], indent=2))
print("\nAnnotation format:")
print(json.dumps(data['annotations'][0], indent=2))

Required GroundingDINO fields:

Image format:
{
  "id": 29,
  "file_name": "10.012_5_slice.png",
  "width": 630,
  "height": 630
}

Annotation format:
{
  "id": 1,
  "image_id": 2,
  "category_id": 2,
  "bbox": [
    173,
    8,
    30,
    30
  ],
  "area": 900,
  "iscrowd": 0
}


In [ ]:
# import json

# # Paths
# input_file = "train_coco.json"
# output_file = "train_coco_cleaned.json"

# # Load dataset
# with open(input_file, 'r') as f:
#     coco_data = json.load(f)

# # Remove "text" key from images
# for image in coco_data["images"]:
#     image.pop("text", None)

# # Remove "text" key from annotations
# for annotation in coco_data["annotations"]:
#     annotation.pop("text", None)

# # Save cleaned dataset
# with open(output_file, 'w') as f:
#     json.dump(coco_data, f, indent=4)

# print(f"Cleaned dataset saved to {output_file}")


Cleaned dataset saved to train_coco_cleaned.json


In [20]:
import json
import os

# Paths
# data_root = "10440_TS_99_9_17042"  # Directory containing images
data_root = "/content/drive/MyDrive/10440_TS_99_9_17042"
annotation_file = "train_coco.json"  # Updated dataset file

# Load dataset
with open(annotation_file, 'r') as f:
    data = json.load(f)

# Check if critical fields are empty
print("Validation Results:")
if not data["images"]:
    print("- 'images' array is empty.")
if not data["annotations"]:
    print("- 'annotations' array is empty.")
if not data["categories"]:
    print("- 'categories' array is empty.")

# Check orphaned annotations
image_ids = {img["id"] for img in data["images"]}
orphan_annotations = [
    ann for ann in data["annotations"] if ann["image_id"] not in image_ids
]
if orphan_annotations:
    print(f"- Orphan annotations found: {len(orphan_annotations)}")
else:
    print("- All annotations have valid 'image_id' references.")

# Check missing images
missing_images = [
    img["file_name"] for img in data["images"]
    if not os.path.exists(os.path.join(data_root, img["file_name"]))
]
if missing_images:
    print(f"- Missing image files: {len(missing_images)}")
else:
    print("- All image files are present.")


Validation Results:
- All annotations have valid 'image_id' references.
- All image files are present.


In [14]:
# ! wget https://download.openmmlab.com/mmdetection/v3.0/grounding_dino/groundingdino_swint_ogc_mmdet-822d7e9d.pth -P checkpoints/
# ! wget https://download.openmmlab.com/mmdetection/v3.0/grounding_dino/grounding_dino_swin-t_finetune_16xb2_1x_coco/grounding_dino_swin-t_finetune_16xb2_1x_coco_20230921_152544-5f234b20.pth -P checkpoints/
! wget https://download.openmmlab.com/mmdetection/v3.0/grounding_dino/groundingdino_swint_ogc_mmdet-822d7e9d.pth -P checkpoints/


--2024-12-13 17:31:37--  https://download.openmmlab.com/mmdetection/v3.0/grounding_dino/groundingdino_swint_ogc_mmdet-822d7e9d.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.82.164, 163.181.82.148, 163.181.82.166, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.82.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691901857 (660M) [application/octet-stream]
Saving to: ‘checkpoints/groundingdino_swint_ogc_mmdet-822d7e9d.pth’

groundingdino_swint 100%[===================>] 659.85M  24.2MB/s    in 27s     

2024-12-13 17:32:04 (24.7 MB/s) - ‘checkpoints/groundingdino_swint_ogc_mmdet-822d7e9d.pth’ saved [691901857/691901857]



In [22]:
import json
from collections import defaultdict

# Load and analyze the annotation file
def analyze_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Count annotations per image
    image_to_anns = defaultdict(int)
    for ann in data['annotations']:
        image_to_anns[ann['image_id']] += 1

    # Basic stats
    total_images = len(data['images'])
    images_with_anns = len(image_to_anns)
    empty_images = total_images - images_with_anns

    print(f"Total images: {total_images}")
    print(f"Images with annotations: {images_with_anns}")
    print(f"Images without annotations: {empty_images}")

    # Distribution of annotations per image
    ann_counts = list(image_to_anns.values())
    if ann_counts:
        print(f"Min annotations per image: {min(ann_counts)}")
        print(f"Max annotations per image: {max(ann_counts)}")
        print(f"Total annotations: {sum(ann_counts)}")

    # Check for potential issues
    for ann in data['annotations']:
        bbox = ann['bbox']
        if any(x < 0 for x in bbox):
            print(f"Warning: Negative bbox coordinates in annotation {ann['id']}")
        if bbox[2] == 0 or bbox[3] == 0:
            print(f"Warning: Zero width/height in annotation {ann['id']}")

# Run analysis
analyze_coco_annotations('/content/train_coco.json')

Total images: 147
Images with annotations: 80
Images without annotations: 67
Min annotations per image: 1
Max annotations per image: 5
Total annotations: 159


In [15]:
import json

def clean_coco_annotations(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Get images that have annotations
    images_with_anns = set()
    valid_annotations = []

    for ann in data['annotations']:
        # Check for valid bbox coordinates
        bbox = ann['bbox']
        if all(x >= 0 for x in bbox) and bbox[2] > 0 and bbox[3] > 0:
            valid_annotations.append(ann)
            images_with_anns.add(ann['image_id'])

    # Keep only images that have valid annotations
    valid_images = [img for img in data['images'] if img['id'] in images_with_anns]

    # Create clean dataset
    clean_data = {
        'images': valid_images,
        'annotations': valid_annotations,
        'categories': data['categories']
    }

    with open(output_file, 'w') as f:
        json.dump(clean_data, f)

    print(f"Original images: {len(data['images'])}")
    print(f"Clean images: {len(valid_images)}")
    print(f"Original annotations: {len(data['annotations'])}")
    print(f"Clean annotations: {len(valid_annotations)}")

# Clean both train and val sets
clean_coco_annotations('/content/train_coco.json', '/content/train_coco_clean.json')
clean_coco_annotations('/content/val_coco.json', '/content/val_coco_clean.json')

Original images: 147
Clean images: 78
Original annotations: 202
Clean annotations: 200
Original images: 37
Clean images: 22
Original annotations: 202
Clean annotations: 200


In [24]:
import json

def check_categories_and_annotations(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    print("Categories:")
    for cat in data['categories']:
        print(f"ID: {cat['id']}, Name: {cat['name']}")

    # Check what category IDs are actually used in annotations
    used_cats = set()
    for ann in data['annotations']:
        used_cats.add(ann['category_id'])

    print("\nCategory IDs used in annotations:", sorted(list(used_cats)))

check_categories_and_annotations('/content/train_coco_clean.json')

Categories:
ID: 1, Name: ferritin_complex
ID: 2, Name: beta_amylase
ID: 3, Name: beta_galactosidase
ID: 4, Name: cytosolic_ribosome
ID: 5, Name: thyroglobulin
ID: 6, Name: virus

Category IDs used in annotations: [1, 2, 3, 4, 5, 6]


In [16]:
import json

def remap_categories(input_train_file, input_val_file, output_train_file, output_val_file):
    # Read both files
    with open(input_train_file, 'r') as f:
        train_data = json.load(f)
    with open(input_val_file, 'r') as f:
        val_data = json.load(f)

    # Create mapping from old to new category IDs
    cat_map = {cat['id']: idx for idx, cat in enumerate(train_data['categories'])}

    # Update training data
    for cat in train_data['categories']:
        cat['id'] = cat_map[cat['id']]
    for ann in train_data['annotations']:
        ann['category_id'] = cat_map[ann['category_id']]

    # Update validation data
    for cat in val_data['categories']:
        cat['id'] = cat_map[cat['id']]
    for ann in val_data['annotations']:
        ann['category_id'] = cat_map[ann['category_id']]

    # Save remapped data
    with open(output_train_file, 'w') as f:
        json.dump(train_data, f, indent=2)
    with open(output_val_file, 'w') as f:
        json.dump(val_data, f, indent=2)

    print("Original category IDs:", sorted(list(cat_map.keys())))
    print("New category IDs:", sorted(list(cat_map.values())))

# Run the remapping
remap_categories(
    '/content/train_coco_clean.json',
    '/content/val_coco.json',
    '/content/train_coco_remapped.json',
    '/content/val_coco_remapped.json'
)

Original category IDs: [1, 2, 3, 4, 5, 6]
New category IDs: [0, 1, 2, 3, 4, 5]


In [17]:
! python /content/drive/MyDrive/mmdetection/tools/analysis_tools/browse_dataset.py /content/drive/MyDrive/finetune_config.py --output-dir inspect_dataset


2024-12-13 17:33:52.680366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 17:33:52.701453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 17:33:52.707945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 17:33:53.994762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may stil

In [ ]:

! python /content/drive/MyDrive/mmdetection/tools/train.py /content/drive/MyDrive/finetune_config.py


Streaming output truncated to the last 5000 lines.
) at 0x7f37611e3a90>]
Number of predictions: 900
Number of ground truths: 1
Cost matrix shape: torch.Size([900, 1])
Sample of cost matrix:
 tensor([[-1.8728],
        [ 4.2051],
        [-1.8567],
        [-1.9099],
        [ 2.7178]])
Contains NaN: tensor(False)
Contains inf: tensor(False)
Pred bboxes: tensor([[102.7913,  97.4329, 141.8173, 135.6463],
        [216.0483,   9.0733, 255.8695,  46.8063],
        [102.5259,  97.8431, 141.3778, 135.9814],
        [104.6924,  98.0456, 142.9929, 135.8190],
        [124.7729,   1.5498, 163.1868,  39.8681]], device='cuda:0')
GT bboxes: tensor([[104.1270,  97.7778, 142.2222, 135.8730]], device='cuda:0')
Number of predictions: 900
Number of ground truths: 2
Cost matrix shape: torch.Size([900, 2])
Sample of cost matrix:
 tensor([[ 9.2958,  7.0578],
        [ 7.2556,  9.8273],
        [ 7.2552,  9.8572],
        [14.6040, -1.7145],
        [ 4.3345, 11.8388]])
Contains NaN: tensor(False)
Contains i

Mounted at /content/drive


In [ ]:
! cat mmdetection/configs/grounding_dino/grounding_dino_swin-t_pretrain_obj365_goldg_cap4m.py

_base_ = [
    '../_base_/datasets/coco_detection.py',
    '../_base_/schedules/schedule_1x.py', '../_base_/default_runtime.py'
]

lang_model_name = 'bert-base-uncased'

model = dict(
    type='GroundingDINO',
    num_queries=900,
    with_box_refine=True,
    as_two_stage=True,
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_mask=False,
    ),
    language_model=dict(
        type='BertModel',
        name=lang_model_name,
        pad_to_max=False,
        use_sub_sentence_represent=True,
        special_tokens_list=['[CLS]', '[SEP]', '.', '?'],
        add_pooling_layer=True,
    ),
    backbone=dict(
        type='SwinTransformer',
        embed_dims=96,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.,
        attn_drop_rate=0.,
  